In [1]:
from itertools import product
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import (MaxAbsScaler, MinMaxScaler,
                                   PolynomialFeatures, RobustScaler,
                                   StandardScaler)
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import (BaseCrossValidator, GridSearchCV, KFold,
                                     RandomizedSearchCV, StratifiedKFold,
                                     check_cv, train_test_split)
from sklearn.linear_model import (ARDRegression, BayesianRidge, ElasticNet,
                                  ElasticNetCV, Lars, Lasso, LassoLars,
                                  LinearRegression, LogisticRegression,
                                  LogisticRegressionCV,
                                  OrthogonalMatchingPursuit, Ridge)
from sklearn.ensemble import (GradientBoostingClassifier,
                              GradientBoostingRegressor,
                              RandomForestClassifier, RandomForestRegressor)
from sklearn.base import BaseEstimator, is_regressor
# import econml
# from econml.orf import DMLOrthoForest
# from econml.metalearners import SLearner, TLearner, XLearner
# from econml.grf import CausalForest
from econml.dr import DRLearner
from econml.dml import CausalForestDML, KernelDML, LinearDML, SparseLinearDML
import sklearn.preprocessing
import sklearn.neural_network
import sklearn.linear_model
import sklearn.ensemble
import sklearn
import argparse
import logging
import os
import pdb
import numpy as np
import pandas as pd
import pickle
import utils
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import time
import warnings

During Cross-fitting

In [3]:
def select_classification_hyperparameters(estimator):
    """
    Returns a hyperparameter grid for the specified classification model type.

    Args:
        model_type (str): The type of model to be used. Valid values are 'linear', 'forest', 'nnet', and 'poly'.

    Returns:
        A dictionary representing the hyperparameter grid to search over.
    """
    if isinstance(estimator, LogisticRegressionCV) or estimator == 'linear':
        # Hyperparameter grid for linear classification model
        return {
            'Cs': [1, 10],
            'max_iter': [25]
        }
    elif isinstance(estimator, RandomForestClassifier) or estimator == 'forest':
        # Hyperparameter grid for random forest classification model
        return {
            'n_estimators': [25],
            'max_depth': [None, 5, 10, 20],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2]
        }
    elif isinstance(estimator, GradientBoostingClassifier) or estimator == 'gbf':
        # Hyperparameter grid for gradient boosting classification model
        return {
            'n_estimators': [100, 500],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7],

        }
    elif isinstance(estimator, MLPClassifier) or estimator == 'nnet':
        # Hyperparameter grid for neural network classification model
        return {
            'hidden_layer_sizes': [(10,), (50,), (100,)],
            'activation': ['relu'],
            'solver': ['adam'],
            'alpha': [0.0001, 0.001, 0.01],
            'learning_rate': ['constant', 'adaptive'],
            'max_iter': [25]
        }
    else:
        warnings.warn("No hyperparameters for this type of model. There are default hyperparameters for LogisticRegressionCV, RandomForestClassifier, MLPClassifier, and the polynomial pipleine", category=UserWarning)
        return {}
        # raise ValueError("Invalid model type. Valid values are 'linear', 'forest', 'nnet', and 'poly'.")

In [4]:
def select_regression_hyperparameters(estimator):
    """
    Returns a dictionary of hyperparameters to be searched over for a regression model.

    Args:
        model_type (str): The type of model to be used. Valid values are 'linear', 'forest', 'nnet', and 'poly'.

    Returns:
        A dictionary of hyperparameters to be searched over using a grid search.
    """
    if  isinstance(estimator, ElasticNetCV) or estimator == 'linear':
        return {
            'l1_ratio': [0.1, 0.5, 0.9],
            'max_iter': [100],
        }
    elif isinstance(estimator, RandomForestRegressor) or estimator == 'forest':
        return {
            'n_estimators': [25],
            'max_depth': [None, 10, 50],
            'min_samples_split': [2, 5, 10],
        }
    elif isinstance(estimator, MLPRegressor) or estimator == 'nnet':
        # Hyperparameter grid for neural network classification model
        return {
            'hidden_layer_sizes': [(10,), (50,), (100,)],
            'alpha': [0.0001, 0.001, 0.01],
            'learning_rate': ['constant', 'adaptive'],
            'max_iter': [25]
        }
    elif isinstance(estimator, GradientBoostingRegressor) or estimator == 'gbf':
        # Hyperparameter grid for gradient boosting regression model
        return {
            'n_estimators': [100],
            'learning_rate': [0.01, 0.1, 1.0],
            'max_depth': [3, 5],
        }
    elif isinstance(estimator, RandomForestRegressor) or estimator == 'forest':
        return {'n_estimators': [10]}
    else:
        warnings.warn("No hyperparameters for this type of model. There are default hyperparameters for ElasticNetCV, RandomForestRegressor, MLPRegressor, and the polynomial pipeline.", category=UserWarning)
        return {}

In [5]:
def find_best_model_hyperparam(X, Y, model, is_discrete):
    if type(model) == str:
        if is_discrete:
            model = utils.select_discrete_estimator(model)
        else:
            model = utils.select_continuous_estimator(model)
    if is_discrete:
        hyperparam_grid_model = select_classification_hyperparameters(model)
    else:
        hyperparam_grid_model = select_regression_hyperparameters(model)
    # print(model)
    grid_search = GridSearchCV(model, hyperparam_grid_model, cv=5, n_jobs=-1)
    grid_search.fit(X, Y.ravel())
    best_params = grid_search.best_params_
    current_best_model = grid_search.best_estimator_
    current_best_score = grid_search.best_score_

    # print(current_best_model, type(current_best_model))
    return current_best_model

In [18]:
#Divide into k folds.
#For each fold, select a different model and find the best hyperparameter 
# get score, mse, time of estimators. Set cv=0 for estimator
def get_models_during_k_folds(X, T, Y, ci_estimator_list, model_y, model_t, model_y_discrete, model_t_discrete):
    k = 4
    cv = KFold(n_splits=k, shuffle=True, random_state=123)

    fold_models = {}
    i = 0
    total_start_time = time.time()
    for train_index, test_index in cv.split(X):
        X_train, T_train, Y_train = X.iloc[train_index], T.iloc[train_index], Y.iloc[train_index]
        X_test, T_test, Y_test = X.iloc[test_index], T.iloc[test_index], Y.iloc[test_index]
        causal_model_score = {}
        causal_model_mse = {}
        causal_model_time = {}
        for ci in ci_estimator_list:
            if model_y_discrete:
                current_model_y = utils.select_discrete_estimator(model_y[i])
            else:
                current_model_y = utils.select_continuous_estimator(model_y[i])
            if model_t_discrete:
                current_model_t = utils.select_discrete_estimator(model_t[i])
            else:
                current_model_t = utils.select_continuous_estimator(model_t[i])


            #find the best hyperparameters for the first stage linear models
            best_model_y = find_best_model_hyperparam(X, Y, current_model_y, model_y_discrete)
            best_model_t = find_best_model_hyperparam(X, T, current_model_t, model_t_discrete)

            # best_hyperparam_model_y, best_hyperparam_model_t
            causal_model = utils.get_estimators(ci, best_model_y, best_model_t)
            start_time = time.time()
            causal_model.fit(Y_train, T_train, X=X_train)
            run_time = time.time() - start_time
            te_pred = causal_model.effect(X_test)
            causal_model_mse[ci] = np.mean((Y_test - te_pred)**2)
            causal_model_time[ci] = run_time
        fold_models[f'fold {i}'] = {'model_y' : best_model_y, 'model_t' : best_model_t, 'Mse' : causal_model_mse, 'Runtime' : causal_model_time}
        i += 1

    total_run_time = time.time() - total_start_time
    return fold_models, total_run_time

In [19]:
#Find best fold in terms of MSE and Runtime for each estimator
def find_best_fold_mse_runtime(fold_models, ci_estimator_list):
    mse_all_estimators = {}
    runtime_all_estimators = {}
    for ci in ci_estimator_list:
        mse_all_estimators[f'{ci}'] = []
        runtime_all_estimators[f'{ci}'] = []
    for k, value in fold_models.items():
        for ci in ci_estimator_list:
            mse_all_estimators[f'{ci}'].append(value['Mse'][ci])
            runtime_all_estimators[f'{ci}'].append(value['Runtime'][ci])

    best_mse_fold = {}
    best_runtime_fold = {}
    for ci in ci_estimator_list:
        best_mse_fold[ci] = np.argmin(mse_all_estimators[f'{ci}'])
        best_runtime_fold[ci] = np.argmin(runtime_all_estimators[f'{ci}'])

    best_models_mse = {}
    best_models_time = {}
    for ci in ci_estimator_list:
        fold_mse = best_mse_fold[ci]
        fold_time = best_runtime_fold[ci]
        best_models_mse[ci] = {'best_model_y' : fold_models[f'fold {fold_mse}']['model_y'], 'best_model_t' : fold_models[f'fold {fold_mse}']['model_t'], 'Mse' : fold_models[f'fold {fold_mse}']['Mse'][ci]}
        best_models_time[ci] = {'best_model_y' : fold_models[f'fold {fold_time}']['model_y'], 'best_model_t' : fold_models[f'fold {fold_time}']['model_t'], 'Runtime' : fold_models[f'fold {fold_time}']['Runtime'][ci]}

    return mse_all_estimators, best_mse_fold, best_models_mse, runtime_all_estimators, best_runtime_fold, best_models_time

In [20]:
data, X, T, Y, true_ITE, true_ATE, true_ATE_stderr, is_discrete = utils.load_ihdp()
# X, X_val, T, T_val, Y, Y_val = train_test_split(X, T, Y, train_size=0.6,shuffle=True, random_state=123)
# X_val, X_test, T_val, T_test, Y_val, Y_test = train_test_split(X_val, T_val, Y_val, train_size=.5, shuffle=True, random_state=123)

In [23]:
ci_estimator_list = ['tl', 'dml', 'kernel_dml', 'CausalForestDML']
model_y = ['linear', 'forest', 'gbf', 'nnet']
model_t = ['linear', 'forest', 'gbf', 'nnet']

In [ ]:
is_meta = False
fold_models, total_run_time = get_models_during_k_folds(X, T, Y, ci_estimator_list, model_y, model_t, model_y_discrete=False, model_t_discrete=True)
print(fold_models, total_run_time)

{'fold 0': {'model_y': ElasticNetCV(l1_ratio=0.9, max_iter=100), 'model_t': LogisticRegressionCV(Cs=1, max_iter=25), 'Mse': {'tl': 7.522415610112231, 'dml': 6.258760351060026, 'kernel_dml': 5.314349707327539, 'CausalForestDML': 6.070481862241069}, 'Runtime': {'tl': 0.1490309238433838, 'dml': 8.739501237869263, 'kernel_dml': 0.24778056144714355, 'CausalForestDML': 0.376253604888916}}, 

'fold 1': {'model_y': RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=25), 'model_t': RandomForestClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=25), 'Mse': {'tl': 8.35734849455551, 'dml': 8.786954144845003, 'kernel_dml': 5.401484369803926, 'CausalForestDML': 6.038643439179678}, 'Runtime': {'tl': 0.09996724128723145, 'dml': 1.182527780532837, 'kernel_dml': 0.2831137180328369, 'CausalForestDML': 0.4049530029296875}}, 

'fold 2': {'model_y': GradientBoostingRegressor(), 'model_t': GradientBoostingClassifier(learning_rate=0.01), 'Mse': {'tl': 8.664446722679891, 'dml': 6.432632858973467, 'kernel_dml': 4.720149305256853, 'CausalForestDML': 5.3016284760187204}, 'Runtime': {'tl': 0.14995050430297852, 'dml': 2.421130418777466, 'kernel_dml': 0.4446289539337158, 'CausalForestDML': 0.5565671920776367}}, 

'fold 3': {'model_y': MLPRegressor(max_iter=25), 'model_t': MLPClassifier(hidden_layer_sizes=(50,), max_iter=25), 'Mse': {'tl': 13.107047259176383, 'dml': 9.850744661505283, 'kernel_dml': 4.446290403577857, 'CausalForestDML': 6.526513696665307}, 'Runtime': {'tl': 0.24723076820373535, 'dml': 0.7107350826263428, 'kernel_dml': 0.418302059173584, 'CausalForestDML': 0.5394127368927002}}} 

95.19353413581848


In [25]:
mse_all_estimators, best_mse_fold, best_models_mse, runtime_all_estimators, best_runtime_fold, best_models_time = find_best_fold_mse_runtime(fold_models, ci_estimator_list)

In [26]:
mse_all_estimators, best_mse_fold, best_models_mse

({'tl': [7.522415610112231,
   8.35734849455551,
   8.664446722679891,
   13.107047259176383],
  'dml': [6.258760351060026,
   8.786954144845003,
   6.432632858973467,
   9.850744661505283],
  'kernel_dml': [5.314349707327539,
   5.401484369803926,
   4.720149305256853,
   4.446290403577857],
  'CausalForestDML': [6.070481862241069,
   6.038643439179678,
   5.3016284760187204,
   6.526513696665307]},
 {'tl': 0, 'dml': 0, 'kernel_dml': 3, 'CausalForestDML': 2},
 {'tl': {'best_model_y': ElasticNetCV(l1_ratio=0.9, max_iter=100),
   'best_model_t': LogisticRegressionCV(Cs=1, max_iter=25),
   'Mse': 7.522415610112231},
  'dml': {'best_model_y': ElasticNetCV(l1_ratio=0.9, max_iter=100),
   'best_model_t': LogisticRegressionCV(Cs=1, max_iter=25),
   'Mse': 6.258760351060026},
  'kernel_dml': {'best_model_y': MLPRegressor(max_iter=25),
   'best_model_t': MLPClassifier(hidden_layer_sizes=(50,), max_iter=25),
   'Mse': 4.446290403577857},
  'CausalForestDML': {'best_model_y': GradientBoostingReg

In [27]:
runtime_all_estimators, best_runtime_fold, best_models_time

({'tl': [0.1490309238433838,
   0.09996724128723145,
   0.14995050430297852,
   0.24723076820373535],
  'dml': [8.739501237869263,
   1.182527780532837,
   2.421130418777466,
   0.7107350826263428],
  'kernel_dml': [0.24778056144714355,
   0.2831137180328369,
   0.4446289539337158,
   0.418302059173584],
  'CausalForestDML': [0.376253604888916,
   0.4049530029296875,
   0.5565671920776367,
   0.5394127368927002]},
 {'tl': 1, 'dml': 3, 'kernel_dml': 0, 'CausalForestDML': 0},
 {'tl': {'best_model_y': RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=25),
   'best_model_t': RandomForestClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=5,
                          n_estimators=25),
   'Runtime': 0.09996724128723145},
  'dml': {'best_model_y': MLPRegressor(max_iter=25),
   'best_model_t': MLPClassifier(hidden_layer_sizes=(50,), max_iter=25),
   'Runtime': 0.7107350826263428},
  'kernel_dml': {'best_model_y': ElasticNetCV(l1_ratio=0.9, max_iter=100),
   'best